In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import preprocessing as p
import math
from scipy.stats import norm

In [6]:
df1 = pd.read_csv("sample/2014_01_01.csv")
df2 = pd.read_csv("sample/2014_01_02.csv")
df3 = pd.read_csv("sample/2014_01_03.csv")

In [7]:
p.to_sec(df1)
p.to_sec(df2)
p.to_sec(df3)
p.to_ID(df1)
p.to_ID(df2)
p.to_ID(df3)

## Similarity Check
### ID-agnostic mode

In [20]:
l = []
l += df1[df1.ID == 1]['sec'].tolist()
l += df2[df2.ID == 1]['sec'].tolist()
l += df3[df3.ID == 1]['sec'].tolist()

In [28]:
l.sort()

In [57]:
DIM = 3

#learned by rl
SIGMA = 5000

In [43]:
n_tuples = []
for i in range(len(l)-DIM):
    n_tuples.append(np.array(l[i:i+DIM]))

In [44]:
Tr = n_tuples[-1]
#편의상 하루 중 가장 마지막 기록으로 설정함.
#추후에 실시간으로 돌릴때에는 최근값으로 다시 설정해야함

In [62]:
predicted = []
for i,val in enumerate(n_tuples[:-1]):
    #maximum norm
    if np.linalg.norm(Tr-val, ord = np.inf) < SIGMA:
        predicted.append(n_tuples[i+1][0])

In [63]:
predicted

[81021.0, 81021.0, 81259.0, 82808.0, 82811.0, 83035.0]

일별 초단위로 계산: 특정 시간대에 대한 특징 반영 가능. 대신 다음날로 넘어가는 부분의 연속성은 파악하기 어렵다

### ID-aware mode
To be continued..

## Predictions Phase

In [74]:
DELTA = 900  #second
EPS_prd = 0.3 #ratio

In [85]:
R_prd = []
tmp = []
for i,val in enumerate(predicted[:-1]):
    tmp.append(val)
    if(predicted[i+1] - val > DELTA):
        #separate
        if len(tmp)/len(predicted) > EPS_prd:
            R_prd.append(np.average(tmp))
        tmp = []
tmp.append(predicted[-1])
if len(tmp)/len(predicted) > EPS_prd:
    R_prd.append(np.average(tmp))

In [86]:
R_prd

[81100.33333333333, 82884.66666666667]

## Duty Cycle adaptation

In [3]:
#persentage
DC_MIN = 2
DC_MAX = 20
DC_DEF = 8

#seconds
L_PRD = 900
L_EXP = 360

#standart deviation
SIGMA_PRD = L_PRD/math.sqrt(8*math.log(DC_MAX/DC_DEF))
SIGMA_EXP = L_EXP/math.sqrt(8*math.log(DC_DEF/DC_MIN))
A_PRD = DC_MAX*SIGMA_PRD*math.sqrt(2*math.pi)
A_EXP = DC_DEF*SIGMA_EXP*math.sqrt(2*math.pi)

#learned by rl
GAMMA = 3

In [7]:
def Seren(t, t_s, t_e):
    n_sub = math.ceil((t_e-t_s)/GAMMA)
    if n_sub <= 1:
        return 0
    R_exp = []
    t_exp = 99999 #some big number
    for i in range(1,n_sub):
        t_tmp = t_s+i*(t_e-t_s)/n_sub
#         R_exp.append(t_tmp)
        if abs(t-t_exp) > (t-t_tmp):
            t_exp = t_tmp
    return A_EXP*norm(t_exp, SIGMA_EXP).pdf(t)

In [8]:
#Adaptive Duty Cycling
def Adaptive_DC(t,t_cur,t_prev,R):
    #t: 듀티사이클을 계산하고 있는 시점(추정)
    #t_cur: current estimated contact time from R_prd
    #t_prev: previous ~~
    if R_prd == []:
        return DC_DEF
    C_prd = A_PRD*norm(t_cur, SIGMA_PRD).pdf(t)
    C_exp = Seren(t, t_prev, t_cur)
    C = C_prd+C_exp
    if C<DC_MIN:
        C = DC_MIN
    return C
    